In [1]:
# torch imports
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40

inference_data:
    split: '?'

log:
    root: '?'
    save_preds: False 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?' 
    num_workers: '?' 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0     
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'min-val-abs_area_estimation_error'

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [5]:
%%yaml experiment_cfg 

## Set the high level name
group: "UVS_ACDC_Pilot"

# subgroup: "Base"
subgroup: "Sweep_Threshold"

inference_data:
    _class: "ese.datasets.Segment2D"
    task: "ACDC/Challenge2017/MRI/2"
    split: "train"
    resolution: 128
    slicing: "midslice"
    version: "v4.2"
    return_data_id: True 
    label: 0

base_model:
    - "/storage/vbutoi/scratch/ESE/training/02_16_23_Universeg-HO_Adrian/20230214_112221-1C7K-2cb972b55f72d56b3842c2d2d8bdd3c6"

# Change this for debugging
dataloader:
    batch_size: 1
    num_workers: 1

<IPython.core.display.Javascript object>

In [6]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])

# For inference experiments, we don't add the date by default.
add_date = True
use_best_models = False 
# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg,
    add_date=add_date,
    use_best_models=use_best_models
)

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/hdd/home/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


MultiBinarySegment2D
ese.datasets.Segment2D


ValueError: STOP

In [ ]:
len(inf_cfgs)

In [ ]:
inf_cfgs

## Running Jobs

In [ ]:
from ese.analysis.run_inference import get_cal_stats

In [ ]:
from ese.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=True,
    gpu='0',
)

In [ ]:
# from ese.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     group="inference",
#     base_cfg=updated_base_cfg,
#     add_date=add_date,
#     exp_cfg=experiment_cfg,
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['2', '3'],
#     # available_gpus=['4', '5'],
#     # available_gpus=['2', '3', '4', '5', '6', '7'],
# )